In [1]:
from numpy import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.integrate import cumtrapz
from LabFuncs import *
from Params import *
from NeutrinoFuncs import *
from WIMPFuncs import *
plt.rcParams['axes.linewidth'] = 2.5
plt.rc('text', usetex=True)
plt.rc('font', family='serif',size=25)


def fEres(E):
    Eres = 20.0*(E/200.0)**0.57
    f = 1.0/(sqrt(2*pi)*Eres)*exp(-(E-Eres)**2.0/(2*Eres**2.0))
    return f

def Smear(E,dR):
    f = fEres(E)
    dR = zeros(shape=size(E))
    for i in range(0,size(E)):
        Eres = 20.0*(E[i]/200.0)**0.57
        f = 1.0/(sqrt(2*pi)*Eres)*exp(-(E-Eres)**2.0/(2*Eres**2.0))
        dRs[i] = trapz(dR*f,E) 
    return dRs
    
# Load neutrino fluxes
NuBG = GetNuFluxes(0.0,Xe131)
E_nu_all = NuBG.Energy
Flux_all = NuBG.Flux
n_nu = NuBG.NumberOfNeutrinos 
Errs = NuBG.Uncertainties
solar = NuBG.SolarLabel


col = cm.Spectral(linspace(0,1,7))
colnu = zeros(shape=(n_nu,4))

tit = r"He:SF$_6$ at 740:20 Torr"
HaloModel = SHM
Loc = Boulby
DM = WIMP(9.0,5.0e-45)
wimplabel = r"DM $m_\chi = 9$ GeV"
E_thresh = 1.0
ymax = 1.0e4
ymin = 1.0e-5

xi_S = 32/(32+6*19)
xi_F = 6*19/(32+6*19)

ne = 2000
E_r = logspace(-3,4,ne)
t = JulianDay(9,12,2018,18.0)*ones(shape=ne)
Exposure = 10*(0.16)*6
xmin = 1.0e-3
xmax = 2500.0


fig = plt.figure(figsize=(13,12))
ax = fig.add_subplot(111)
col = cm.jet(linspace(0,1,5))
colnu = zeros(shape=(n_nu,4))
colnu[0,:] = col[0,:]
colnu[1,:] = col[0,:]
colnu[2,:] = col[0,:]
colnu[3,:] = col[0,:]
colnu[4,:] = col[0,:]
colnu[5,:] = col[0,:]
colnu[6,:] = col[0,:]
colnu[7,:] = col[0,:]
colnu[8,:] = col[0,:]
colnu[9,:] = col[1,:]
colnu[10,:] = col[2,:]
colnu[11,:] = col[3,:]
colnu[12,:] = col[3,:]
colnu[13,:] = col[3,:]
colnu[14,:] = col[4,:]
lsty = array(['-','--','-.','-','--','-.','-','--','-.','-','-','-','-','-','-'])

flav_Solar = array([0.55,0.45/2.0,0.45/2.0,0.0,0.0,0.0])
flav_Atm = array([0.17453365,0.33393192,0.0,0.15677881,0.33475563,0.0])
flav_DSNB = array([0.50633522,0.04746695,0.04746695,0.30379697,0.04746695,0.04746695])
flav_Geo = array([0.0,0.0,0.0,0.55,0.45/2.0,0.45/2.0])
flav_Reactor = array([0.0,0.0,0.0,0.55,0.45/2.0,0.45/2.0])
flav_SN_11 = array([0.41825567,0.07184627,0.07184627,0.29435926,0.07184627,0.07184627])
flav_SN_27 = array([0.30680873,0.1147821,0.1147821,0.23403129,0.11479789,0.11479789])

def dRdEe_nu(E_r,t,sol,E_nu,Flux,Nuc,flav):
    N = Nuc.NumberOfNeutrons
    Z = Nuc.NumberOfProtons
    Q_W = 1.0*N-(1-4.0*sinTheta_Wsq)*Z # weak nuclear hypercharge
    m_N_GeV = 0.93141941*(N+Z) # nucleus mass in GeV
    m_N_keV = m_N_GeV*1.0e6 # nucleus mass in keV
    m_e_GeV = 5.109989461e-4

    ne = size(E_r)

    fMod = 1.0

    gV = array([2*sinTheta_Wsq+0.5,2*sinTheta_Wsq-0.5,2*sinTheta_Wsq-0.5,
               2*sinTheta_Wsq+0.5,2*sinTheta_Wsq-0.5,2*sinTheta_Wsq-0.5])
    gA = array([0.5,-0.5,-0.5,-0.5,0.5,0.5])

    T = E_r/1000 # MeV
    Tmax = 2*E_nu**2.0/(2*E_nu+m_e_GeV*1000)

    dR = 0.0
    for ii in arange(0,6)[flav>0]:
        dRdE = zeros(shape=ne)
        As = (gV[ii]+gA[ii])**2.0
        Bs = (gV[ii]-gA[ii])**2.0
        Cs = (gA[ii]**2.0-gV[ii]**2.0)
        if Flux[1]>0.0:
            for i in range(0,ne):
                diff_sigma = (G_F_GeV**2.0*m_e_GeV/(2*pi))*(As+Bs*(1-T[i]/E_nu)**2.0+Cs*1000*m_e_GeV*T[i]/E_nu**2.0)
                diff_sigma *= (0.197e-13)**2.0*(1.0e-6)*1000.0/(1.0*N+1.0*Z)*(N_A)
                diff_sigma[T[i]>Tmax] = 0.0
                dRdE[i] = trapz(diff_sigma*Flux,x=E_nu)
        else:
            diff_sigma = (G_F_GeV**2.0*m_e_GeV/(2*pi))*(As+Bs*(1-T/E_nu[0])**2.0+Cs*1000*m_e_GeV*T/E_nu[0]**2.0)
            diff_sigma *= (0.197e-13)**2.0*(1.0e-6)*1000.0/(1.0*N+1.0*Z)*(N_A)
            diff_sigma[T>Tmax[0]] = 0.0
            dRdE = diff_sigma*Flux[0]*E_nu[0]*(diff_sigma>0.0) # for monochromatic nu's
            
        # Convert into /ton/year/keV
        dRdE = fMod*dRdE*(365.0*3600.0*24.0*1000.0)
        dRdE = dRdE*Z*flav[ii]

        dR += dRdE
    
    return dR

plt.fill(array([xmin,xmax,xmax,0.0,0.0,xmin]),array([ymin,ymin,1.0,1.0,ymax,ymax]),color='Gray',alpha=0.2,edgecolor="None")



# Solar
errs = array([0.006,0.01,0.3,0.04,0.04,0.06,0.3,0.4,0.5,\
             0.2,\
             0.5,\
             0.3,0.3,0.3,\
             0.3])
n = 1000
Phi_u = zeros(shape=ne-1)
Phi_l = zeros(shape=ne-1)
Phi_2u = zeros(shape=ne-1)
Phi_2l = zeros(shape=ne-1)
Phi1 = zeros(shape=ne-1)
for i in range(0,9):
    R = xi_F*dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],F19,flav_Solar)+xi_S*dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],S32,flav_Solar)+dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],He4,flav_Solar)
    Ep = E_r[R>0.0]
    R = R[R>0.0]
    Phi1 = zeros(shape=ne-1)
    for j in range(0,size(R)-1):
        Phi1[j] = sum(Exposure*0.5*(Ep[(j+1):]-Ep[j:-1])*(R[(j+1):]+R[j:-1]))
    Phi_u += Phi1*(1+errs[i])
    Phi_l += Phi1*(1-errs[i])
    Phi_2u += Phi1*(1+2*errs[i])
    Phi_2l += Phi1*(1-2*errs[i])
Phi_u[Phi_u==0.0] = ymin/10.0
Phi_l[Phi_l==0.0] = ymin/10.0
Phi_2u[Phi_2u==0.0] = ymin/10.0
Phi_2l[Phi_2l==0.0] = ymin/10.0
x = (E_r[1:]+E_r[0:-1])/2
plt.fill_between(x,Phi_2u,y2=Phi_2l,color=colnu[0,:],alpha=0.2,linewidth=3,edgecolor=colnu[0,:],zorder=1)
plt.fill_between(x,Phi_u,y2=Phi_l,color=colnu[0,:],alpha=0.4,label="Solar",linewidth=3,edgecolor=colnu[0,:],zorder=1)
plt.plot(x,Phi_u,lsty[0],color=colnu[0,:],linewidth=3,zorder=1) 
plt.plot(x,Phi_l,lsty[0],color=colnu[0,:],linewidth=3,zorder=1) 

# Geo
GeoU = loadtxt('../../data/neutrinos/geoneutrinos/GeoU-Boulby.txt')
GeoTh = loadtxt('../../data/neutrinos/geoneutrinos/GeoTh-Boulby.txt')
GeoK = loadtxt('../../data/neutrinos/geoneutrinos/GeoK-Boulby.txt')
GeoErrs = array([0.75/3.24,0.75/3.16,3.0/13.44]) # from https://arxiv.org/pdf/1301.0365.pdf
E_nu_geo = GeoU[:,0]
FluxGeo = zeros(shape=(1000,3))
FluxGeo[:,0] = GeoU[:,1]
FluxGeo[:,1] = GeoTh[:,1]
FluxGeo[:,2] = GeoK[:,1]
Phi_geo_u = zeros(shape=ne-1)
Phi_geo_l = zeros(shape=ne-1)
Phi_geo_2u = zeros(shape=ne-1)
Phi_geo_2l = zeros(shape=ne-1)
for i in range(0,3):
    R = xi_F*dRdEe_nu(E_r,t,solar[i],E_nu_geo,FluxGeo[:,i],F19,flav_Geo)+xi_S*dRdEe_nu(E_r,t,solar[i],E_nu_geo,FluxGeo[:,i],S32,flav_Geo)+dRdEe_nu(E_r,t,solar[i],E_nu_geo,FluxGeo[:,i],He4,flav_Geo)
    Ep = E_r[R>0.0]
    R = R[R>0.0]
    Phi1 = zeros(shape=ne-1)
    for j in range(0,size(R)-1):
        Phi1[j] = sum(Exposure*0.5*(Ep[(j+1):]-Ep[j:-1])*(R[(j+1):]+R[j:-1]))
    Phi_geo_u += Phi1*(1+GeoErrs[i])
    Phi_geo_l += Phi1*(1-GeoErrs[i])
    Phi_geo_2u += Phi1*(1+2*GeoErrs[i])
    Phi_geo_2l += Phi1*(1-2*GeoErrs[i])
Phi_geo_u[Phi_geo_u==0.0] = ymin/10.0
Phi_geo_l[Phi_geo_l==0.0] = ymin/10.0
Phi_geo_2u[Phi_geo_2u==0.0] = ymin/10.0
Phi_geo_2l[Phi_geo_2l==0.0] = ymin/10.0
x = (E_r[1:]+E_r[0:-1])/2
plt.fill_between(x,Phi_geo_2u,y2=Phi_geo_2l,color=colnu[11,:],alpha=0.2,linewidth=3,edgecolor=colnu[11,:],zorder=2)
plt.fill_between(x,Phi_geo_u,y2=Phi_geo_l,color=colnu[11,:],alpha=0.4,label="Geo",linewidth=3,edgecolor=colnu[11,:],zorder=2)
plt.plot(x,Phi_geo_u,lsty[0],color=colnu[11,:],linewidth=3,zorder=2) 
plt.plot(x,Phi_geo_l,lsty[0],color=colnu[11,:],linewidth=3,zorder=2) 

# Reactor
i = 14
Reactor_err = 0.1
Reactor = loadtxt('../../data/neutrinos/ReactorAntiNu-Boulby.txt')
R = xi_F*dRdEe_nu(E_r,t,False,Reactor[:,0],Reactor[:,1],F19,flav_Reactor)+xi_S*dRdEe_nu(E_r,t,False,Reactor[:,0],Reactor[:,1],S32,flav_Reactor)+dRdEe_nu(E_r,t,False,Reactor[:,0],Reactor[:,1],He4,flav_Reactor)
Ep = E_r[R>0.0]
R = R[R>0.0]
N = zeros(shape=(size(R)-1))
for j in range(0,size(R)-1):
    N[j] = sum(Exposure*0.5*(Ep[(j+1):]-Ep[j:-1])*(R[(j+1):]+R[j:-1]))
x = append(Ep[1:],Ep[-1])
y = append(N,0.0)
x = append(xmin,x)
y = append(N[0],y)
y1 = y*(1+2*Reactor_err)
y2 = y*(1-2*Reactor_err)
y2[y2<0.0] = ymin/10.0
plt.fill_between(x,y1,y2=y2,color=colnu[i,:],alpha=0.2,linewidth=3,edgecolor=colnu[i,:],zorder=3)
y1 = y*(1+Reactor_err)
y2 = y*(1-Reactor_err)
plt.fill_between(x,y1,y2=y2,color=colnu[i,:],alpha=0.4,label=nuname[i],linewidth=3,edgecolor=colnu[i,:],zorder=3)
plt.plot(x,y2,lsty[i],color=colnu[i,:],linewidth=3,zorder=3)
plt.plot(x,y1,lsty[i],color=colnu[i,:],linewidth=3,zorder=3)


# Atm
i = 10
R = xi_F*dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],F19,flav_Atm)+xi_S*dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],S32,flav_Atm)+dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],He4,flav_Atm)
Ep = E_r[R>0.0]
R = R[R>0.0]
N = zeros(shape=(size(R)-1))
for j in range(0,size(R)-1):
    N[j] = sum(Exposure*0.5*(Ep[(j+1):]-Ep[j:-1])*(R[(j+1):]+R[j:-1]))
x = append(Ep[1:],Ep[-1])
y = append(N,0.0)
y1 = y*(1+2*errs[i])
y2 = y*(1-2*errs[i])
y2[y2<=0.0] = ymin/10.0
plt.fill_between(x,y1,y2=y2,color=colnu[i,:],alpha=0.2,linewidth=3,edgecolor=colnu[i,:],zorder=5)
y1 = y*(1+errs[i])
y2 = y*(1-errs[i])
plt.fill_between(x,y1,y2=y2,color=colnu[i,:],alpha=0.4,label=nuname[i],linewidth=3,edgecolor=colnu[i,:],zorder=5)
plt.plot(x,y2,lsty[i],color=colnu[i,:],linewidth=3,zorder=5)
plt.plot(x,y1,lsty[i],color=colnu[i,:],linewidth=3,zorder=5)


# DSNB
i = 9
R = xi_F*dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],F19,flav_DSNB)+xi_S*dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],S32,flav_DSNB)+dRdEe_nu(E_r,t,solar[i],E_nu_all[:,i],Flux_all[:,i],He4,flav_DSNB)
Ep = E_r[R>0.0]
R = R[R>0.0]
N = zeros(shape=(size(R)-1))
for j in range(0,size(R)-1):
    N[j] = sum(Exposure*0.5*(Ep[(j+1):]-Ep[j:-1])*(R[(j+1):]+R[j:-1]))
x = append(Ep[1:],Ep[-1])
y = append(N,0.0)
x = append(xmin,x)
y = append(N[0],y)
y[-1] = 0.0
y1 = y*(1+2*errs[i])
y2 = y*(1-2*errs[i])
#y2[y2<0.0] = ymin/10.0
plt.fill_between(x,y1,y2=y2,color=colnu[i,:],alpha=0.3,linewidth=3,edgecolor=colnu[i,:],zorder=6)
y1 = y*(1+errs[i])
y2 = y*(1-errs[i])
plt.fill_between(x,y1,y2=y2,color=colnu[i,:],alpha=0.4,label=nuname[i],linewidth=3,edgecolor=colnu[i,:],zorder=6)
plt.plot(x,y2,lsty[i],color=colnu[i,:],linewidth=3,zorder=6)
plt.plot(x,y1,lsty[i],color=colnu[i,:],linewidth=3,zorder=6)

distance = 8.0
data1 = loadtxt('../../data/neutrinos/supernovae/SNburst_11.2Mo.txt')
data2 = loadtxt('../../data/neutrinos/supernovae/SNburst_27Mo.txt')
E_nu_SN = data1[:,0]
Flux_SN1 = (10.0/distance)**2.0*data1[:,1]/(6*365*3600*24)
Flux_SN2 = (10.0/distance)**2.0*data2[:,1]/(6*365*3600*24)
R1 = xi_F*dRdEe_nu(E_r,t,False,E_nu_SN,Flux_SN1,F19,flav_SN_11)+xi_S*dRdEe_nu(E_r,t,False,E_nu_SN,Flux_SN1,S32,flav_SN_11)+dRdEe_nu(E_r,t,False,E_nu_SN,Flux_SN1,He4,flav_SN_11)
R2 = xi_F*dRdEe_nu(E_r,t,False,E_nu_SN,Flux_SN2,F19,flav_SN_27)+xi_S*dRdEe_nu(E_r,t,False,E_nu_SN,Flux_SN2,S32,flav_SN_27)+dRdEe_nu(E_r,t,False,E_nu_SN,Flux_SN2,He4,flav_SN_27)
Ep = E_r[R1>0.0]
R2 = R2[R1>0.0]
R1 = R1[R1>0.0]
N1 = zeros(shape=(size(R1)-1))
N2 = zeros(shape=(size(R1)-1))
for j in range(0,size(R1)-1):
    N1[j] = sum(Exposure*0.5*(Ep[(j+1):]-Ep[j:-1])*(R1[(j+1):]+R1[j:-1]))
    N2[j] = sum(Exposure*0.5*(Ep[(j+1):]-Ep[j:-1])*(R2[(j+1):]+R2[j:-1]))
x = append(Ep[1:],Ep[-1])
y1 = append(N1,ymin)
y2 = append(N2,ymin)
plt.fill_between(x,y1,y2=y2,facecolor='Purple',alpha=0.5,linewidth=3,zorder=5)
plt.plot(x,y1,color='Purple',linewidth=3,zorder=5)
plt.plot(x,y2,color='Purple',linewidth=3,zorder=5)


plt.xlabel(r"$E_{\rm threshold}$ [keV]",fontsize=35)
plt.ylabel(r"Number of Events in 10,000 m$^{3}$ in 6 years",fontsize=35)        
ax.set_xlim(left=xmin, right=xmax)
ax.set_ylim(bottom=ymin, top=ymax)
ax.tick_params(which='major',direction='in',width=2,length=13,right=True,top=True,pad=7,labelsize=30)
ax.tick_params(which='minor',direction='in',width=1,length=10,right=True,top=True)
#leg = plt.legend(loc='upper right',fontsize=20,edgecolor='k')
#leg.get_frame().set_linewidth(1.5)
plt.gcf().text(0.55,0.82,tit,fontsize=35)
plt.title(r'{\bf Electron recoils}',fontsize=40)

plt.grid()

offs = (1-0.02)
plt.text(750,2.3e2,r'{\bf Solar}',color=col[0,:],fontsize=30,rotation=-12)

plt.text(75*offs,8e-4,r'{\bf DSNB}',color='k',fontsize=30,rotation=0,zorder=10)
plt.text(75,8e-4*offs,r'{\bf DSNB}',color=col[1,:],fontsize=30,rotation=0,zorder=10)

plt.text(100*offs,5.5e-5,r'{\bf Atmospheric}',color='k',fontsize=30,rotation=-1,zorder=10)
plt.text(100,5.5e-5*offs,r'{\bf Atmospheric}',color='green',fontsize=30,rotation=-1,zorder=10)

plt.text(75*offs,7.3e-1,r'{\bf Geo}',color='k',fontsize=30,rotation=-12,zorder=10)
plt.text(75,7.3e-1*offs,r'{\bf Geo}',color=col[3,:],fontsize=30,rotation=-12,zorder=10)

plt.text(75*offs,1.5e1,r'{\bf Reactor}',color='k',fontsize=30,rotation=-10,zorder=10)
plt.text(75,1.5e1*offs,r'{\bf Reactor}',color=col[4,:],fontsize=30,rotation=-10,zorder=10)

#plt.text(1.3e-3,1.5e3,r'{\bf WIMP} $m_\chi = 9$ GeV',color='k',fontsize=28,rotation=0)

plt.text(100*offs,0.7e-1,r'{\bf 8 kpc CCSN}',color='k',fontsize=30,rotation=-2,zorder=10)
plt.text(100,0.7e-1*offs,r'{\bf 8 kpc CCSN}',color='Purple',fontsize=30,rotation=-2,zorder=10)

plt.yscale('log')

plt.show()
pltname = "NeutrinoRates_Threshold_electron"
fig.savefig('../../plots/'+pltname+'.pdf',bbox_inches='tight')
fig.savefig('../../plots/plots_png/'+pltname+'.png',bbox_inches='tight')

IndexError: index 11 is out of bounds for axis 0 with size 11

In [2]:
n_nu

11